In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import skimage
import pandas as pd
import anndata as ad
from collections import namedtuple
import networkx as nx
import math
import sys
import getopt
import anndata as ad
from voronoi import *


In [2]:
class snakemake():
    input = ['/gpfs/commons/home/jsingh/results/test_anndata.h5ad', '/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/seurat_analysis/pyseq_test_types.csv'
            ,'/gpfs/commons/home/jsingh/results/uns.csv','/gpfs/commons/home/jsingh/results/centroid.npy']
    output = ['']
    

In [3]:
adata =  ad.read_h5ad(snakemake.input[0])
with open(snakemake.input[3], 'rb') as f:
    adata.uns['centroid'] = np.load(f,allow_pickle=True)

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
x_coord = []
y_coord = []
for x in range(np.shape(adata.uns['centroid'])[0]):
    y_coord.append(adata.uns['centroid'][x][0])
    x_coord.append(adata.uns['centroid'][x][1])

In [5]:
Point = namedtuple('Point', ['x','y'])
point_list = []
for i,y in zip(x_coord,y_coord):
    point_list.append(Point(i,y))
v,l,e = computeVoronoiDiagram(point_list)

In [65]:
#Building Graph
gr = nx.Graph()
for pt in v:
    gr.add_node(pt)
    
edge_list= []
for ei in range(len(e)):
    edge_list.append(e[ei][1:])
    
for ed in edge_list:
    gr.add_edge(ed[0],ed[1])

In [66]:
df = pd.read_csv('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/seurat_analysis/pyseq_test_types.csv')
cell_type = {}
for i,ty in zip(df.id,df.type):
    cell_type.update({i: ty})

In [67]:
def get_neighbors(graph, node_val, rad):
    subgraph = nx.ego_graph(G = graph, n= node_val, radius=rad, center=True, undirected=False, distance=None)
    node_list = list(subgraph.nodes())
    return(node_list)
    

In [68]:
def get_cell_type (node_list):
    cell_frame = pd.DataFrame()
    cell_list = []
    for n in node_list:
        cell_list.append(cell_type[n])
    cell_frame['id'] = node_list
    cell_frame['type'] = cell_list
   
    return cell_frame 
        

In [69]:
def cell_pixel_intensity_gfap(frame):
    intensity_list = []
    for idx in frame.id:
        intensity_list.append(mean_intensities[idx - 1]) #minus 1 as we removed intensity for label 0
        
    frame['gfap_intensity'] = intensity_list
    
    
    return frame
        
        
    


In [70]:
neigh = get_cell_type(get_neighbors(gr,node_val = 10, rad = 4))
neigh['type'].value_counts(normalize=True)

KeyError: 0

In [64]:
neigh = get_cell_type(get_neighbors(gr,node_val = 10, rad = 4))
neigh['type'].value_counts(normalize=True)

34
0
Pvalb
Pvalb
Pvalb
Oligo
Oligo
Oligo
L2/3 IT
Pvalb
Pvalb
Oligo
Pvalb
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Pvalb
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
Oligo
-1
32
31


ValueError: Length of values (31) does not match length of index (32)

In [71]:
def get_nodes(dt):
    selected_nodes = []
    for k in dt.keys():
        if dt[k] == 2:
            selected_nodes.append(k)
    return(selected_nodes)

In [12]:
def neighbors(graph,node,limit):
    spatial_dict = nx.single_source_shortest_path_length(graph, node, cutoff=limit)
    return spatial_dict
  
#get_nodes(neighbors(gr,4,2))    

In [73]:
neighbor_dict = {}

In [74]:
for g in range(gr.number_of_nodes()):
   
    try:
        neighbor_dict.update({g:get_nodes(neighbors(gr,g,2))})
        
    except: 
        pass
    

In [75]:
neighbor_dict

{0: [2, 13, 16, 17, -1],
 1: [3, 100, 135, 200, 9, 10, 11, 12, 205, 14, 208, 209, 82, 179, 19, 23, 25],
 2: [0, 7, 13, 18, -1],
 3: [1, 100, 135, 200, 8, 11, 205, 46, 208, 209, 82, 179, 19, 23, 25, 28],
 4: [39, 15, 48, 28, -1],
 5: [9, 15, 20, 26],
 6: [15, 18, 24, 27],
 7: [2, 13, 18, 21, 29],
 8: [3, 18, 20, 28],
 9: [1, 5, 10, 22, 26],
 10: [1, 9, 12, 21, 22],
 11: [33, 1, 3, 100, 135, 200, 205, 14, 208, 209, 82, 179, 23, 25],
 12: [32, 1, 10, 14, 21],
 13: [0, 2, 37, 7, 17, 22],
 14: [32, 1, 33, 11, 12],
 15: [4, 5, 6, 36, 46, -1],
 16: [32, 0, 34, 17, 19, -1],
 17: [0, 33, 37, 13, 16],
 18: [2, 6, 7, 8, 24, 29],
 19: [1, 3, 100, 35, 135, 200, 205, 16, 208, 82, 179, 209, 25],
 20: [36, 5, 8, 26, 30, 31],
 21: [32, 7, 10, 43, 12, 29],
 22: [37, 38, 9, 10, 13, 26],
 23: [1, 34, 3, 100, 135, 200, 11, 205, 78, 208, 209, 82, 179, 25],
 24: [6, 40, 18, 27, 29, 31],
 25: [1, 3, 100, 135, 200, 41, 11, 205, 46, 208, 209, 82, 179, 19, 23],
 26: [5, 38, 9, 20, 22, 30],
 27: [6, 54, 24, 59, 2